In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive')
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/final/'
sys.path.append(project_dir+'services')

Mounted at /content/drive


In [2]:
%%writefile "/content/drive/MyDrive/Colab Notebooks/project/final/services/logging_service.py"
"""
Logging service for the music recommender app.
Records user activity and listening history.
"""
import os
import pandas as pd
from datetime import datetime

class LoggingService:
    """Service for logging user activity."""

    def __init__(self, history_csv_path):
        """
        Initialize the logging service.

        Args:
            history_csv_path (str): Path to the listening history CSV file
        """
        self.history_csv_path = history_csv_path
        self.load_history()

    def load_history(self):
        """Load listening history from CSV."""
        if os.path.exists(self.history_csv_path):
            self.history_df = pd.read_csv(self.history_csv_path)
        else:
            self.history_df = pd.DataFrame(columns=[
                'history_id', 'user_id', 'song_id', 'emotion_before',
                'emotion_after', 'timestamp', 'is_recommended'
            ])

    def save_history(self):
        """Save listening history to CSV."""
        self.history_df.to_csv(self.history_csv_path, index=False)

    def log(self, user_id, song_id, emotion_before, emotion_after, is_recommended=0):
        """
        Log a song play event.

        Args:
            user_id (int): The user ID
            song_id (int): The song ID
            emotion_before (str): Emotion before playing the song
            emotion_after (str): Emotion after playing the song
            is_recommended (int): Whether the song was recommended (1) or not (0)

        Returns:
            int: The new history record ID
        """
        # Generate a new history ID
        if self.history_df.empty:
            new_history_id = 1
        else:
            new_history_id = self.history_df['history_id'].max() + 1

        # Create the new history record
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        new_record = pd.DataFrame([{
            'history_id': new_history_id,
            'user_id': user_id,
            'song_id': song_id,
            'emotion_before': emotion_before,
            'emotion_after': emotion_after,
            'timestamp': timestamp,
            'is_recommended': is_recommended
        }])

        # Append to history DataFrame
        self.history_df = pd.concat([self.history_df, new_record], ignore_index=True)
        self.save_history()

        return new_history_id

    def get_user_history(self, user_id):
        """
        Get a user's listening history.

        Args:
            user_id (int): The user ID

        Returns:
            pandas.DataFrame: The user's listening history
        """
        if self.history_df.empty:
            return pd.DataFrame()

        return self.history_df[self.history_df['user_id'] == user_id]

    def get_song_play_count(self, song_id):
        """
        Get the number of times a song has been played.

        Args:
            song_id (int): The song ID

        Returns:
            int: The number of times the song has been played
        """
        if self.history_df.empty:
            return 0

        return len(self.history_df[self.history_df['song_id'] == song_id])

    def get_most_played_songs(self, limit=10):
        """
        Get the most played songs.

        Args:
            limit (int): Maximum number of songs to return

        Returns:
            list: List of song IDs sorted by play count
        """
        if self.history_df.empty:
            return []

        # Count plays for each song
        song_counts = self.history_df['song_id'].value_counts()

        # Sort by play count and get top songs
        top_songs = song_counts.head(limit).index.tolist()

        return top_songs

Overwriting /content/drive/MyDrive/Colab Notebooks/project/final/services/logging_service.py
